![book header](header.png)

In [2]:
import time
from pynput import keyboard
import numpy as np
import matplotlib.pyplot as plt
# from serial import Serial # Uncomment this line if you are using the real car
from KITT_Simulator.serial_simulator import Serial # Uncomment this line if you are using the simulator

# Module 1 - Connecting with and Controlling KITT

In this module we will start working with the car simulator and the car.
We will start with providing an overview and a brief description on getting to know the vehicle and implementing some basic control scripts.



## Brief Overview

KITT is a small vehicle that you can control remotely, in manyways similar to a typical RC car. While traditional RC cars usually come with a joystick for control, this project has adapted the car so that you can use your computer instead. So for the manual control instead of using a joystick, you control the car with your keyboard, where pressing different keys can make it move forward, backward, or turn.

However, manual control is not the main focus of this project. As you gain experience in sending commands and receiving data from the car, you will be able to write a program that automates KITT's movements, eliminating the need for manual control.

For this remote control system to function, KITT needs to be wirelessly connected to your computer. This wireless connection allows you to send commands to KITT, such as controlling its direction or speed. Additionally, KITT can send information back to your computer, such as sensor data or battery level, which is a key difference from conventional RC cars. So this setup is a **two-way** communication link.

As the first part, you will learn how to set up this wireless connection and start controlling KITT using basic commands. This will build a foundation for more advanced tasks, such as programming you PC and connect it with KITT to navigate it autonomously.

### KITT Basics

KITT is built on a modified toy car model, the Traxxas E-MAXX (see picture below). In addition to its original components, KITT is equipped with two ultrasonic sensors, an LCD display, and the necessary electronics to transmit data via a Bluetooth connection. The LCD on the car provides real-time information, such as the readings from the distance measurement sensors and the battery voltage level.

**Note:** There is a data sheet as well which gives an overview of the various components, such as the motor controller, LCD status indicator, ultrasonic sensors, and, most importantly, the communication module.

**Note:** As a heads-up for the future, when you start gathering data from KITT via the wireless link, you might notice that it doesn't always match what’s shown on the car's LCD. The data received directly from the car is more accurate, as among other reasons the LCD might update at a slower rate. Therefore, while the LCD can serve as an initial indicator of any issues, it's best to rely on the direct data from the car for precise information.

![Traxass E-MAXX along with it's dimensions](traxxas_e-maxx.png)

*Traxass E-MAXX along with it's dimensions*

#### Hardware

KITT is equipped with a custom-made circuit board that brings together various components needed for its operation. At the heart of this board is a microcontroller, which acts as the main control unit.

This board also includes everything necessary for wireless communication, like the Bluetooth module, as well as connectors for other parts of the car. Additionally, there's an amplifier on the board that's used for sending out audio signals that help with localization.

The circuit board is powered by rechargeable batteries. These batteries supply the necessary voltage through a component that adjusts the voltage to the required levels.

Here are the key components included on the board:

- **Microcontroller:** NXP LPC4357 chip with an ARM CORTEX-M4/M0 core
- **Bluetooth module**
- **Amplifier for the audio beacon**
- **Buck converter** for voltage regulation
- **Voltage regulator:** LM1117-3.3 for generating 3.3V DC


![Schematic of communication with KITT and PC.](communication_overview.png)

*Schematic of communication with KITT and PC.*


As KITT has a bluethoth module for communication you can use your computer's internal Bluetooth connector to connect to KITT.

The Bluetooth communication with KITT consists of the following elements:

-  On car: Roving Networks RN-41 I/RM, Onboard Bluetooth module with UART control.
-  On PC: LM Technologies LM506, USB Bluetooth V4.0 dongle with Broadcom BCM20702 chipset, or the internal Bluetooth module of your laptop.

**Tip:** If you want to use one of the lab computers or have trouble connecting to KITT from your PC (check if all drivers are installed), you can use the USB Bluetooth dongle.

KITT has four LED's on the front, see the figure below, their meaning is as follows:

![KITT](figurejoin.png)

*Kitt control board and front LEDS*

- Red (twice): *5V* and *20V* supply voltages are present.
- Green (blinking): Bluetooth searches for connection; (steady) Bluetooth connected.
- Yellow (blinking): Bluetooth data transfer.

The button next to the LEDs is a reset button for the MCU. This reset button allows the user to reset KITT's actions if it freezes or is unresponsive.

## Communicating with KITT

In this section, we cover the essential concepts required to effectively communicate with the KITT system. In the following section, we will apply these principles in practice, implementing them in the simulator and the car.



#### pySerial

pySerial is a Python module that provides a simple and efficient way to communicate with serial ports. It allows Python programs to access and manipulate the serial ports on a computer, allowing them to communicate with other devices connected to those ports. With pySerial, you can easily send and receive data to/from these devices. pySerial is cross-platform, and works on Windows, macOS, and Linux operating systems.

- `Serial(port, baud rate)` - This command initializes a serial connection. The port argument specifies the serial port to use (e.g., "COM1" on Windows or "/dev/rfcomm0" on Linux),
and the baud-rate argument specifies the data rate in bits per second.
- `serial.write(data)` - This command sends data over the serial connection. The data argument is a bytes object that contains the data to be sent.
- `serial.read(size)` - This command reads a specified number of bytes from the serial connection. It blocks until the specified number of bytes is received.
- `serial.read_until(bytes)` - This command reads bytes from the serial connection until the specified byte sequence is found. It blocks until then.
- `serial.flush()` - This command is used to flush the input and output buffers of the serial connection.
- `serial.close()` - This command is used to close the serial connection.

Further more, updated information regarding the module can also be found through this link [pySerial](https://pyserial.readthedocs.io/en/latest/pyserial.html).

#### Simulated Serial port

The KITT simulator is also controlled by a serial port.
This port uses the following functions:

- `Serial(port, baud rate)`
- `serial.write(data)`

These functions work identically to the pySerial functions, except that they communicate with the simulated KITT and not the real KITT.

#### Controlling KITT

After connecting to KITT, commands are required to control it: Driving commands, Audio beacon commands, Status command.

__Driving Commands:__ First off, for driving there are 2 types of instructions:

-  A direction command: `D`.
-  A motor speed command: `M`.

These are controlled using Pulse width modulation.  Both commands are neutral at a setting of 150. The direction commands range from 200 (hard left) to 100 (hard right), and the motor commands range from 135 (backward) to 165 (forward).

**Note:** There is a dead zone for the motor commands, so KITT will likely not start moving forward until the PWM is set to about 153. It is recommended to experiment with these values, they are also battery-dependent. You should test the size of the drive command dead-band, verify that 150 is the middle position for steering (sometimes there is a deviation of ±2), and test the maximum left and right position.

All the commands are sent in binary and end with a new line character.
```{code-cell}
serial.write(b'code\n')
```

To set direction to 130 and motor speed to 160, it can be done as follows,

```{code-cell}
serial.write(b'D130\n')
serial.write(b'M160\n')
```

**Attention:** Once you set the motor speed commands, KITT will continue to act on this until you either
transmit a new motor speed command or reset the MCU using the button on KITT, which will set both
direction and motor speed commands to the neutral value of 150. Do one of these two options after your
tests since KITT will keep driving if you don’t.

Let's see this in the simulator.

In [3]:
# Open serial port
serial = Serial('/dev/ttyUSB0', 115200)

# Wait for one second
time.sleep(1)

# Set speed and direction
serial.write(b'M162\n')
serial.write(b'D100\n')
print("Motors are ON")

time.sleep(2)

# Set speed to zero
serial.write(b'M150\n')
print("Motors are OFF")

time.sleep(5) # Notice the car keeps moving

# Close the connection (important!)
serial.close()

Canvas(height=520, width=520)

Motors are ON
Motors are OFF


To test yourself, try to make the car turn left instead of right.

Also notice that the car keeps moving after the motors are off. This is because of the car's dynamics, which will be explored in a later module.

__Audio Beacon__

Secondly, Audio beacon commands used are:

- `F`: set transmission frequency (Timer 0)
- `B`: set bit frequency (Timer 1)
- `R`: set repetition count (Timer 3)
- `C`: set 32-bit code
- `A`: turn beacon on/off

![Illustration of the transmitted signal](Beacon.png)
*Illustration of the transmitted signal*

The repitition count formula is as follows:

$$Repitition Count = \frac{Bit Frequency}{Repitition Frequency}$$

For example, if the bit frequency is 5000, the repetition count should be 2500 to sound the beacon twice per second.

Later, you will use this beacon to determine where your car is on the field.
Remember that all the numbers provided here serve only as an example. It is up to you to determine what carrier frequency, code word, etc., best fits the goal of succeeding at the final challenge.

Let's use these in the simulator.
Unfortunately, the sound of the car cannot be heard in the simulation.
Therefore, nothing happens.

In [4]:
serial = Serial('/dev/ttyUSB0', 115200)

# Carrier frequency set to 10000 Hz for example.
carrier_frequency = (10000).to_bytes(2, byteorder='big') # Convert number to bytes
serial.write(b'F' + carrier_frequency + b'\n') # `F` command sets carrier frequency

# Bit frequency set to 5000 Hz for example.
bit_frequency = (5000).to_bytes(2, byteorder='big')
serial.write(b'B' + bit_frequency + b'\n') # `B` command sets bit frequency

# Repitition count has a minimum of 32.
repetition_count = (2500).to_bytes(2, byteorder='big')
serial.write(b'R' + repetition_count + b'\n') # `R` command sets repetition count

# The 32 bits code pattern is transmitted bit-wise over the beacon. Hence, code must be specified in hexadecimal.
# The following code is a sequence of 16 `1` bits followed by 16 `0` bits
# You will need to create your own code later
code = (0xFFFF0000).to_bytes(4, byteorder='big') # Convert hexadecimal number to bytes
serial.write(b'C' + code + b'\n') # `C` command sets code

time.sleep(1)
print("Beacon is ON")
serial.write(b'A1\n') # Turn Beacon ON
time.sleep(3)
print("Beacon is OFF")
serial.write(b'A0\n') # Turn Beacon OFF
time.sleep(1)

# Close the connection (important!)
serial.close()

Canvas(height=520, width=520)

Beacon is ON
Beacon is OFF


**Caution:** Be aware that the default code word for the beacon is 0x00000000, which means KITT will not start making noise on its own when the beacon is turned on. You should specify a code as described above before you can hear the beacon make noise. Furthermore, an arbitrary carrier frequency, bit frequency and repitition count can be used. (Generally, maximum of 30 kHz)

__status command__

Lastly, the status command which requests for the status and all the data from KITT is `S`.
The status string reports the current drive commands, the ultrasonic sensor distance (cm), the battery voltage (mV), the audio beacon status (on/off) and control parameters (code word,carrier-frequency, bit-frequency, repetition count). A sensor distance of 999 means overload (i.e., out of range).

In [6]:
serial = Serial('/dev/ttyUSB0', 115200)

serial.write(b'S\n')

status = serial.read_until(b'\x04')

print(f"Car status is:\n\n{status}")

# Close the connection (important!)
serial.close()

Canvas(height=520, width=520)

Car status is:

**************************
* Audio Beacon: off
* c: 0xabcdef00
* f_c: 5678
* f_b: 1234
* c_r: 1337
**************************
* PWM:
* Dir. 150
* Mot. 150
**************************
* Sensors:
* Dist. L 999 R 999
* V_batt 11.5 V
**************************



## Implementing a basic KITT Control Script

Now you can start to write a Python program to control the simulator or the car connected over Bluetooth. This program should allow the car to be controlled using the keyboard w, a, s, and d keys. The e and q should start and stop the audio beacon. Also add a key to stop communication with KITT.

We start this program by defining a the class KITT and some essential methods for communication. Classes define the structure and capabilities of objects, allowing you to create multiple instances with the same properties and methods. Next you will complete the *wasd* function. You have to design the wasd function as a continuous loop that reads keyboard events using the keyboard library which you load using import keyboard. You will finally call the function and test your code first on the simulator and next on the real car.

Here is a short explanation of the code that you will complete in the following code cells. Ensure you understand how the code works before using its various functions.

**KITT Class:***
- The `__init__` method is the one that runs when you make an instance of the class. It starts the
serial communication with the specified port and baud rate.
- `send_command` method sends a given command to KITT over the established serial connection.
- `set_speed` and `set_angle` methods adjust KITT’s speed and steering angle, respectively.
- `stop` method brings KITT to a halt by setting both speed and angle to a neutral state.
- `__del__` method ensures the proper closure of the serial communication when the KITT object is
deleted. It runs automatically at the end of the script.

**wasd Function:**
- The wasd function is designed to be a continuous loop that reads keyboard events using
    the keyboard library which you loaded using import keyboard.
- When a key is pressed (KEY_DOWN), the function interprets the key and adjusts KITT’s speed and steering angle or toggles the beacon accordingly.
- The ‘w’ key accelerates KITT forward, ‘s’ stops KITT, ’a’ turns KITT left, and ‘d’ turns KITT right.
- The ‘e’ key turns on the beacon, and the ‘q’ key turns off the beacon.
- When a key is released (KEY_UP), you could define appropriate actions, e.g. stop KITT or reset the steering angle.

**test code**
- `if __name__ == "__main__"` This is the code that will start executing when you run your script. This contstruct allows you to
mport your script as a module into another script without running the test code.

- An instance of KITT (an object of the KITT class) is then created, with the port as a parameter.
You should change this to your port number.
- Next, the wasd function is called and given this instance of KITT. It should detect key-press events
and attach these to KITT commands like
– kitt.set_speed(170)
– kitt.set_angle(150)
– kitt.stop

**TODO:** modify the __init__ so that when the communication with KITT is started, the beacon
is initialized with the correct set of parameters. Use the existing serial port and send_command.

**TODO:** add two methods start_beacon and stop_beacon that turn the beacon on or off. Note
that you should have set the beacon parameters during the __init__, so there is no need to resend
them every time you turn the beacon on.

In [7]:
class KITT:
    def __init__(self, port, baudrate=115200):
        self.serial = Serial(port, baudrate, rtscts=True)

    def send_command(self, command):
        self.serial.write(command.encode())

    def set_speed(self, speed):
        self.send_command(f'M{speed}\n')

    def set_angle(self, angle):
        self.send_command(f'D{angle}\n')

    def stop(self):
        self.set_speed(150)
        self.set_angle(150)

    def __del__(self):
        self.serial.close()


In [8]:
def wasd_control(kitt):
    print("Control the car with E (up), D (down), S (left), F (right), and Q to quit.")
    car_speed = 150
    car_steering = 150

    def on_press(key):
        nonlocal car_speed, car_steering

        try:
            if key.char == 'e':
                car_speed += 5
            elif key.char == 'd':
                car_speed -= 5
            elif key.char == 's':
                car_steering += 15
            elif key.char == 'f':
                car_steering -= 15
            elif key.char == 'q':
                kitt.__del__()
                return False
        except AttributeError:
            pass
        
        if car_speed > 165:
            car_speed = 165
        if car_speed < 135:
            car_speed = 135
        if car_steering > 200:
            car_steering = 200
        if car_steering < 100:
            car_steering = 100
        
        kitt.set_speed(car_speed)
        kitt.set_angle(car_steering)
        # print(f"Speed: {car_speed}, Steering: {car_steering}")

    # Collect events until released
    with keyboard.Listener(on_press=on_press) as listener:
        listener.join()



In [9]:
# Example usage
if __name__ == "__main__":
    kitt = KITT('/dev/ttyUSB0')  # Replace with the actual port
    wasd_control(kitt)

Canvas(height=520, width=520)

Control the car with E (up), D (down), S (left), F (right), and Q to quit.


#### To test the code with the real car

Now you can check and see if you can also control the real car with the code you wrote. For that you need to restart the kernel and uncomment the serial at the top of the notebook. No code changes required.

#### Connecting to the real KITT

Although the code does not need to be changed to work on the real car, theory and practice may differ.
If you encounter any difficulties with connecting to KITT, read on.

To start with, connect the car to your computer over Bluetooth. Make sure you know the name of the Bluetooth port that the car is connected to. To access the link in Python use,
```{code-cell}
serial_port = Serial(comport, 115200, rtscts=True)
```
serial_port is an instance of pySerial and this object can be used to manipulate KITT. The variable comport is the path to the comport assigned to the Bluetooth connection with KITT. Which
comport specifically KITT is connected to can be found in your communication settings in Windows etc. .

E.g., if your transmission connection to KITT takes place over port 10, then you should use:
`comport = 'COM10'`

Connecting to a serial port over Bluetooth can be pretty tricky. Windows users, refer to Appendix B. Linux users can use `# rfcomm bind rfcomm0 MAC_ADDRESS`; there is also a section about it on the [Arch Linux wiki](  https://wiki.archlinux.org/title/Bluetooth#Bluetooth_serial). The MAC address of KITT always ends with the code printed on its label. For example, if the label reads DA:84, the MAC will end with `xx:xx:xx:xx:DA:84`.

To find out which serial ports are available, you can use the list_ports tool of pySerial. Run the following in a shell:

```{code-cell}
python -m serial.tools.list_ports
```

Further information regarding the issues with connecting to the port via Bluetooth can be solved by referring to the Appendices.

**Note:** Also keep in mind, The Bluetooth connection is disturbed by leaving comports open, quitting your work without closing the communication link, removing the Bluetooth dongle, or turning off the Bluetooth connection. Avoid disturbing the Bluetooth connection by ending the connection properly before doing any of the aforementioned things. If you disturb the Bluetooth link, you may need to reboot your computer to reset the operating system, which costs you valuable lab time.

The connection must also be closed once you are done using the port
`serial_port.close()`

#### Optional Extensions

If you would like to go above and beyond with the task then below listed are a few tips which can help with extending the functionality of the program,

- Add error handling: Currently, if there is an error with the Bluetooth connection or the serial communication, the program will simply crash. You could add some error handling to handle
these cases more gracefully, such as printing an error message and exiting the program.
- Add speed control: Currently, the program only supports moving the car forward or backward and turning left or right. You could add support for controlling the speed of the car, such as by
sending different commands to the car depending on how long the user holds down the forward or backward keys.
- Add an emergency stop: KITT doesn’t have a brake; the 150 speed setting lets KITT roll out to standstill, which might take a long distance. Add an emergency brake by letting KITT drive
backwards for a short period of time. (You should first detect if the previous speed setting caused KITT to move forward. You need to define a state variable to memorize the speed setting.)



#### Mid-term assessment and report
After you finish this assignment, and ultimo in week 4, showcase the functionality of your script to your
assigned TA. After you pass this assessment, you are ready to document your results in your midterm
report.
For this Module, you would include a chapter that covers the approach, implementation, testing and
results of the basic controller.
Remember to document your code, using comments to define input/output variables of functions and to
explain the logic and any modifications made. Your completed script will be crucial for the upcoming
challenges, contributing to the overall autonomous driving system.
After you have completed this module and have tested all the components thoroughly, start on the second
part of the communication script outlined in Module 2.

# **FAQ**



**What is a serial communication and why is it used?**

Serial communication is a way of transmitting data one bit at a time over a single communication line. It's commonly used to connect devices like computers and microcontrollers because it simplifies wiring and is efficient for long-distance communication. In this method, data is sent in a sequence, or "serially," from the sender to the receiver, typically using standards like USB, RS-232, or UART.




**What is the COM port?**

In a Bluetooth connection, a COM port (short for "communication port") is a virtual port that allows your computer to communicate with Bluetooth devices as if they were connected via a traditional serial port (like those used for older wired connections). When you pair a Bluetooth device with your computer, the operating system assigns it a virtual COM port, which acts as an interface for data exchange between your computer and the Bluetooth device. This makes it easier to send and receive data over Bluetooth using standard serial communication methods.


**What is a buck converter and why is it used?**

A buck converter is a type of DC-DC converter that steps down voltage from a higher level to a lower level while maintaining efficient power transfer. It works by rapidly switching a transistor on and off to control the energy transferred to an output capacitor, which smooths out the voltage to a lower, steady level.

Buck converters are used because they are highly efficient, meaning they waste very little power as heat. This efficiency is particularly important in battery-powered devices, like KITT, where it's crucial to conserve energy and maintain a stable voltage for sensitive electronics. For instance, if the battery supplies 12V but the circuit requires 5V, a buck converter efficiently reduces the voltage to the needed level without generating much heat or wasting energy.




**I have paired the car with my PC however the LED is still bliking is the car connected ?**

Pairing your Bluetooth module and assigning a COM port is different from running the Python script. If you only pair the device, you'll notice that the LED continues to blink after some time. However, when you run the Python script and the virtual COM port is properly allocated, the LED will stop blinking. Simply pairing the device isn't enough to stop the LED from blinking—you need to run the Python script as instructed to see the LED would be stable which means there is an actualy connection.

**What is pulse width modulation?**

Pulse Width Modulation (PWM) is a technique used to control the amount of power delivered to an electronic device by varying the width of the pulses in a signal. It works by switching a signal on and off rapidly at a constant frequency, and by adjusting the ratio of the "on" time to the "off" time, you can control the average voltage and power delivered to a load.

For example, if a signal is "on" for 50% of the time and "off" for the other 50%, the output will deliver 50% of the maximum power. This method is commonly used in applications like controlling the speed of motors, dimming LED lights, and generating analog signals from digital sources because it is efficient and provides precise control over power.


**I've implemented the code, and everything functions correctly in the GUI and runs without errors, but the car isn't moving. What should I do?**

Keep in mind that there's an additional switch that controls the power to the motors. If this switch is turned off, the motors won't receive any power from the battery, even if everything else is set up correctly, and the car won't move. Make sure this switch is turned on.